Requirements
numpy==1.19.5
pandas==1.0.1
scikit-learn==0.22.1
scipy==1.4.1
joblib==0.14.1
mlflow==1.6.0
matplotlib==3.7.0
scikit-learn==1.2.1


In [2]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

from joblib import dump,load





In [3]:
df = pd.read_csv('../data.csv', delimiter = ',')
print(df.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 148670 entries, 0 to 148669
Data columns (total 34 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   ID                         148670 non-null  int64  
 1   year                       148670 non-null  int64  
 2   loan_limit                 145326 non-null  object 
 3   Gender                     148670 non-null  object 
 4   approv_in_adv              147762 non-null  object 
 5   loan_type                  148670 non-null  object 
 6   loan_purpose               148536 non-null  object 
 7   Credit_Worthiness          148670 non-null  object 
 8   open_credit                148670 non-null  object 
 9   business_or_commercial     148670 non-null  object 
 10  loan_amount                148670 non-null  int64  
 11  rate_of_interest           112231 non-null  float64
 12  Interest_rate_spread       112031 non-null  float64
 13  Upfront_charges            10

In [4]:
# Seleciona as variáveis independentes
independentcols = ['income', 'age', 'year', 'Credit_Score','Gender']
independentcols = ['income', 'age', 'year','Gender']

# Normaliza o campo age com numeros 
le_age = LabelEncoder()
le_gender = LabelEncoder()
df['age'] = le_age.fit_transform(df['age'])
df['Gender'] = le_gender.fit_transform(df['Gender'])

# Dropa do dataset os valores nulos das variáveis indepententes
df = df.dropna(subset=independentcols)

# X recebe somente as variáveis independentes do treinamento do cluster
X = df[independentcols]


In [5]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 139520 entries, 0 to 148669
Data columns (total 4 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   income  139520 non-null  float64
 1   age     139520 non-null  int32  
 2   year    139520 non-null  int64  
 3   Gender  139520 non-null  int32  
dtypes: float64(1), int32(2), int64(1)
memory usage: 4.3 MB


In [6]:
# Normalizar as variáveis independentes
scaler = StandardScaler()
X_norm = scaler.fit_transform(X)

In [7]:
# Escolha o número de clusters
kmeans = KMeans(n_clusters=6)
kmeans.fit(X_norm)

C:\Users\leovi\AppData\Roaming\Python\Python311\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


KMeans(n_clusters=6)

In [8]:
# Adiciona o cluster como uma nova coluna no dataframe
df['Cluster'] = kmeans.labels_
X['Cluster'] = kmeans.labels_

C:\Users\leovi\AppData\Local\Temp\ipykernel_7132\3008842561.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['Cluster'] = kmeans.labels_


In [9]:
y = df['Status']

In [10]:
X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=.3, random_state=10)

In [11]:

# Cria modelo de regressão logística para prever a inadimplência
X_cluster = pd.get_dummies(X_train['Cluster'], prefix='Cluster')
X_all = pd.concat([X_train, X_cluster], axis=1)
model = LogisticRegression()
model.fit(X_all, y_train)

LogisticRegression()

In [12]:
X_cluster_test = pd.get_dummies(X_test['Cluster'], prefix='Cluster')
X_all_test = pd.concat([X_test, X_cluster_test], axis=1)
y_pred = model.predict_proba(X_all_test)[:, 1]
# y_pred
print(len(y_test))
print(len(y_pred))


41856
41856


In [13]:
# Avalia o modelo
from sklearn.metrics import accuracy_score

auc = roc_auc_score(y_test, y_pred)
# acc = accuracy_score(y_test, y_pred)

print (y_pred)
print (y_test)

print('AUC:', auc)

[0.2797334  0.23161439 0.22276859 ... 0.22889772 0.25085201 0.20033694]
122888    1
155       1
66574     0
90773     1
113912    1
         ..
92638     0
22400     0
82690     0
78766     0
103365    0
Name: Status, Length: 41856, dtype: int64
AUC: 0.5779193210691784


In [14]:
# # Carrega o modelo
# dump(model,'modelo.joblib')

# Carrega o cluster
dump(kmeans,'cluster.joblib')

# Carrega o labelencoder
dump(le_age, 'labelencoder_age.joblib')
dump(le_gender, 'labelencoder_gender.joblib')

# Carrega o scaler de normalização
dump(scaler, 'scaler.joblib')

['scaler.joblib']

In [15]:
print(le_age.transform(['55-64']))
print(le_age.inverse_transform([3]))




[3]
['55-64']


In [16]:

df.groupby('Cluster')['Status'].mean()

Cluster
0    0.264404
1    0.208608
2    0.246458
3    0.372881
4    0.304962
5    0.224107
Name: Status, dtype: float64

In [17]:

dff = df.groupby('Cluster')['Status']

In [18]:
le_gender.classes_

array(['Female', 'Joint', 'Male', 'Sex Not Available'], dtype=object)

In [19]:
le_age.classes_

array(['25-34', '35-44', '45-54', '55-64', '65-74', '<25', '>74', nan],
      dtype=object)

In [20]:
df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 139520 entries, 0 to 148669
Data columns (total 35 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   ID                         139520 non-null  int64  
 1   year                       139520 non-null  int64  
 2   loan_limit                 136272 non-null  object 
 3   Gender                     139520 non-null  int32  
 4   approv_in_adv              138643 non-null  object 
 5   loan_type                  139520 non-null  object 
 6   loan_purpose               139394 non-null  object 
 7   Credit_Worthiness          139520 non-null  object 
 8   open_credit                139520 non-null  object 
 9   business_or_commercial     139520 non-null  object 
 10  loan_amount                139520 non-null  int64  
 11  rate_of_interest           104120 non-null  float64
 12  Interest_rate_spread       104120 non-null  float64
 13  Upfront_charges            10

In [26]:
df.groupby('Cluster')['income'].mean()
# Grupo 3 alta renda masculino



Cluster
0      6257.538131
1      7076.861907
2      5512.474506
3    166865.084746
4      5139.279425
5     22934.758929
Name: income, dtype: float64

In [31]:
print(le_age.classes_)
print(df.groupby('Cluster')['age'].median())
print(df.groupby('Cluster')['age'].mean())
# Grupo 2 e 4 inicio da 3° idade com média renda 
# Grupo 0 e 2° idade com méda reda 


['25-34' '35-44' '45-54' '55-64' '65-74' '<25' '>74' nan]
Cluster
0    1.0
1    1.0
2    3.0
3    2.0
4    3.0
5    2.0
Name: age, dtype: float64
Cluster
0    1.165025
1    1.161807
2    3.746400
3    2.288136
4    3.729953
5    2.149107
Name: age, dtype: float64


In [40]:
print(le_gender.classes_)
counts = df.groupby('Cluster')['Gender'].value_counts()
print(counts)
# Grupo 4 sem definição de sexo

['Female' 'Joint' 'Male' 'Sex Not Available']
Cluster  Gender
0        2         24294
         3         17536
1        1         21034
         0         14051
2        1         14198
         0         11985
3        2            27
         1            23
         3             7
         0             2
4        3         16245
         2         13398
5        1          3174
         2          2066
         3          1146
         0           334
Name: Gender, dtype: int64


In [45]:
counts = df.groupby('Cluster')['Status'].value_counts()
print(counts)
# Grupo 5, 0  de alta propensão

Cluster  Status
0        0         30770
         1         11060
1        0         27766
         1          7319
2        0         19730
         1          6453
3        0            37
         1            22
4        0         20603
         1          9040
5        0          5214
         1          1506
Name: Status, dtype: int64


In [50]:
counts = df.groupby('Cluster')['loan_amount'].median()
print(counts)
counts = df.groupby('Cluster')['loan_amount'].mean()
print(counts)


Cluster
0    306500.0
1    356500.0
2    256500.0
3    556500.0
4    226500.0
5    486500.0
Name: loan_amount, dtype: float64
Cluster
0    328008.486732
1    372400.812313
2    285631.115609
3    754296.610169
4    263314.087643
5    563004.464286
Name: loan_amount, dtype: float64
